<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Same_Day_Line_Disconnects_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [127]:
# Read a comma-separated values (csv) file into a Pandas DataFrame.
fact_table = pd.read_csv('drive/My Drive/Churn/discos_device_grp_act_vol_invol_same_day_aug.csv')

In [128]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,vol_port_basic_cdma,vol_port_basic_non_cdma,vol_port_sp_android_other,vol_port_sp_apple,vol_non_port_basic_cdma,vol_non_port_basic_non_cdma,vol_non_port_sp_android_other,vol_non_port_sp_apple,invol_basic_cdma,invol_basic_non_cdma,invol_sp_android_other,invol_sp_apple,vol_port_tablet,vol_non_port_tablet,invol_tablet,vol_port_watch,vol_non_port_watch,invol_watch,vol_port_mbb,vol_non_port_mbb,invol_mbb,vol_port_hpc,vol_non_port_hpc,invol_hpc,vol_port_hum,vol_non_port_hum,invol_hum,vol_port_gizmo,vol_non_port_gizmo,invol_gizmo,vol_port_other,vol_non_port_other,invol_other
0,12/1/2016,218,NaN,961,1034,1415,NaN,1502,1152,411,NaN,2126,1731,15.0,2650,1409,NaN,25,38,1.0,655,322,29.0,213,119,3.0,106,99,2.0,41,51,NaN,13,8
1,12/2/2016,197,NaN,1192,1117,1367,NaN,1989,1076,269,NaN,1491,1312,38.0,2803,901,1.0,18,26,4.0,695,169,35.0,185,51,NaN,73,72,1.0,50,30,3.0,25,1
2,12/3/2016,268,NaN,1357,1566,921,NaN,846,784,263,NaN,1728,1370,34.0,2018,1301,3.0,13,32,8.0,465,311,17.0,124,79,1.0,52,75,1.0,58,31,1.0,8,6
3,12/4/2016,239,NaN,925,1356,534,NaN,593,525,239,NaN,1226,1007,23.0,1393,1067,1.0,12,23,1.0,265,237,9.0,73,78,NaN,27,59,NaN,38,33,1.0,10,1
4,12/5/2016,198,NaN,1088,1154,1533,NaN,1412,1129,307,NaN,1960,1456,27.0,2482,1317,2.0,32,23,5.0,630,249,33.0,198,78,NaN,84,81,2.0,66,41,NaN,14,7


In [129]:
# Split out indivudal datasets for forecasting.
# Phones - Basic CDMA
vol_port_basic_cdma = fact_table[['ds', 'vol_port_basic_cdma']].rename(columns = {'vol_port_basic_cdma': 'y'})
vol_non_port_basic_cdma = fact_table[['ds', 'vol_non_port_basic_cdma']].rename(columns = {'vol_non_port_basic_cdma': 'y'})
invol_basic_cdma = fact_table[['ds', 'invol_basic_cdma']].rename(columns = {'invol_basic_cdma': 'y'})

# Phones - Basic Non CDMA
vol_port_basic_non_cdma = fact_table[['ds', 'vol_port_basic_non_cdma']].rename(columns = {'vol_port_basic_non_cdma': 'y'})
vol_non_port_basic_non_cdma = fact_table[['ds', 'vol_non_port_basic_non_cdma']].rename(columns = {'vol_non_port_basic_non_cdma': 'y'})
invol_basic_non_cdma = fact_table[['ds', 'invol_basic_non_cdma']].rename(columns = {'invol_basic_non_cdma': 'y'})

# Phones - Android & Other
vol_port_sp_android_other = fact_table[['ds', 'vol_port_sp_android_other']].rename(columns = {'vol_port_sp_android_other': 'y'})
vol_non_port_sp_android_other = fact_table[['ds', 'vol_non_port_sp_android_other']].rename(columns = {'vol_non_port_sp_android_other': 'y'})
invol_sp_android_other = fact_table[['ds', 'invol_sp_android_other']].rename(columns = {'invol_sp_android_other': 'y'})

# Phones - Apple
vol_port_sp_apple = fact_table[['ds', 'vol_port_sp_apple']].rename(columns = {'vol_port_sp_apple': 'y'})
vol_non_port_sp_apple = fact_table[['ds', 'vol_non_port_sp_apple']].rename(columns = {'vol_non_port_sp_apple': 'y'})
invol_sp_apple = fact_table[['ds', 'invol_sp_apple']].rename(columns = {'invol_sp_apple': 'y'})

# Tablets
vol_port_tablet = fact_table[['ds', 'vol_port_tablet']].rename(columns = {'vol_port_tablet': 'y'})
vol_non_port_tablet = fact_table[['ds', 'vol_non_port_tablet']].rename(columns = {'vol_non_port_tablet': 'y'})
invol_tablet = fact_table[['ds', 'invol_tablet']].rename(columns = {'invol_tablet': 'y'})

# Wearables - Watches
vol_port_watch = fact_table[['ds', 'vol_port_watch']].rename(columns = {'vol_port_watch': 'y'})
vol_non_port_watch = fact_table[['ds', 'vol_non_port_watch']].rename(columns = {'vol_non_port_watch': 'y'})
invol_watch = fact_table[['ds', 'invol_watch']].rename(columns = {'invol_watch': 'y'})

# Other - MBB
vol_port_mbb = fact_table[['ds', 'vol_port_mbb']].rename(columns = {'vol_port_mbb': 'y'})
vol_non_port_mbb = fact_table[['ds', 'vol_non_port_mbb']].rename(columns = {'vol_non_port_mbb': 'y'})
invol_mbb = fact_table[['ds', 'invol_mbb']].rename(columns = {'invol_mbb': 'y'})

# Other - HPC
vol_port_hpc = fact_table[['ds', 'vol_port_hpc']].rename(columns = {'vol_port_hpc': 'y'})
vol_non_port_hpc = fact_table[['ds', 'vol_non_port_hpc']].rename(columns = {'vol_non_port_hpc': 'y'})
invol_hpc = fact_table[['ds', 'invol_hpc']].rename(columns = {'invol_hpc': 'y'})

# Other - Hum
vol_port_hum = fact_table[['ds', 'vol_port_hum']].rename(columns = {'vol_port_hum': 'y'})
vol_non_port_hum = fact_table[['ds', 'vol_non_port_hum']].rename(columns = {'vol_non_port_hum': 'y'})
invol_hum = fact_table[['ds', 'invol_hum']].rename(columns = {'invol_hum': 'y'})

# Other - Gizmo
vol_port_gizmo = fact_table[['ds', 'vol_port_gizmo']].rename(columns = {'vol_port_gizmo': 'y'})
vol_non_port_gizmo = fact_table[['ds', 'vol_non_port_gizmo']].rename(columns = {'vol_non_port_gizmo': 'y'})
invol_gizmo = fact_table[['ds', 'invol_gizmo']].rename(columns = {'invol_gizmo': 'y'})

# Other - Other
vol_port_other = fact_table[['ds', 'vol_port_other']].rename(columns = {'vol_port_other': 'y'})
vol_non_port_other = fact_table[['ds', 'vol_non_port_other']].rename(columns = {'vol_non_port_other': 'y'})
invol_other = fact_table[['ds', 'invol_other']].rename(columns = {'invol_other': 'y'})

In [130]:
# Check the individual dataset format (testing only)
vol_port_tablet.head()

,ds,y
0,12/1/2016,15.0
1,12/2/2016,38.0
2,12/3/2016,34.0
3,12/4/2016,23.0
4,12/5/2016,27.0


In [131]:
# Define the holidays
new_years_day = pd.DataFrame({
    'holiday': 'new_years_day',
    'ds': pd.to_datetime(['2017-01-01',
                          '2018-01-01',
                          '2019-01-01',
                          '2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

mlk_day = pd.DataFrame({
    'holiday': 'mlk_day',
    'ds': pd.to_datetime(['2017-01-16',
                          '2018-01-15',
                          '2019-01-21',
                          '2020-01-20']),
    'lower_window': 0,
    'upper_window': 0,
})

easter_sunday = pd.DataFrame({
    'holiday': 'easter_sunday',
    'ds': pd.to_datetime(['2017-04-16',
                          '2018-04-01',
                          '2019-04-21',
                          '2020-04-12']),
    'lower_window': 0,
    'upper_window': 0,
})

memorial_day = pd.DataFrame({
    'holiday': 'memorial_day',
    'ds': pd.to_datetime(['2017-05-29',
                          '2018-05-28',
                          '2019-05-27',
                          '2020-05-25']),
    'lower_window': 0,
    'upper_window': 0,
})

independence_day = pd.DataFrame({
    'holiday': 'independence_day',
    'ds': pd.to_datetime(['2017-07-04',
                          '2018-07-04',
                          '2019-07-04',
                          '2020-07-04']),
    'lower_window': 0,
    'upper_window': 0,
})

labor_day = pd.DataFrame({
    'holiday': 'labor_day',
    'ds': pd.to_datetime(['2017-09-04',
                          '2018-09-03',
                          '2019-09-02',
                          '2020-09-07']),
    'lower_window': 0,
    'upper_window': 0,
})

thanksgiving_day = pd.DataFrame({
    'holiday': 'thanksgiving_day',
    'ds': pd.to_datetime(['2017-11-23',
                          '2018-11-22',
                          '2019-11-28',
                          '2020-11-26']),
    'lower_window': 0,
    'upper_window': 0,
})

black_friday = pd.DataFrame({
    'holiday': 'black_friday',
    'ds': pd.to_datetime(['2017-11-24',
                          '2018-11-23',
                          '2019-11-29',
                          '2020-11-27']),
    'lower_window': 0,
    'upper_window': 0,
})

cyber_monday = pd.DataFrame({
    'holiday': 'cyber_monday',
    'ds': pd.to_datetime(['2017-11-27',
                          '2018-11-26',
                          '2019-12-02',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

christmas_day = pd.DataFrame({
    'holiday': 'christmas_day',
    'ds': pd.to_datetime(['2017-12-25',
                          '2018-12-25',
                          '2019-12-25',
                          '2020-12-25']),
    'lower_window': 0,
    'upper_window': 0,
})

In [132]:
holidays = pd.concat((new_years_day,
                      mlk_day,
                      easter_sunday,
                      memorial_day,
                      independence_day,
                      labor_day,
                      thanksgiving_day,
                      black_friday,
                      cyber_monday,
                      christmas_day))

In [133]:
# Define the models
# Phones - Basic CDMA
vol_port_basic_cdma_model = Prophet(holidays = holidays,
                                    changepoint_prior_scale = 0.1,
                                    seasonality_prior_scale = 0.01)
vol_non_port_basic_cdma_model = Prophet(holidays = holidays,
                                        changepoint_prior_scale = 0.01,
                                        seasonality_prior_scale = 0.01)
invol_basic_cdma_model = Prophet(holidays = holidays,
                                 changepoint_prior_scale = 0.5,
                                 seasonality_prior_scale = 0.01)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_model = Prophet(holidays = holidays,
                                        changepoint_prior_scale = 0.1,
                                        seasonality_prior_scale = 1.0)
vol_non_port_basic_non_cdma_model = Prophet(holidays = holidays,
                                            changepoint_prior_scale = 0.1,
                                            seasonality_prior_scale = 1.0)
invol_basic_non_cdma_model = Prophet(holidays = holidays,
                                     changepoint_prior_scale = 0.1,
                                     seasonality_prior_scale = 10.0)

# Phones - Android & Other
vol_port_sp_android_other_model = Prophet(holidays = holidays,
                                          changepoint_prior_scale = 0.01,
                                          seasonality_prior_scale = 0.01)
vol_non_port_sp_android_other_model = Prophet(holidays = holidays,
                                              changepoint_prior_scale = 0.5,
                                              seasonality_prior_scale = 10.0)
invol_sp_android_other_model = Prophet(holidays = holidays,
                                       changepoint_prior_scale = 0.5,
                                       seasonality_prior_scale = 0.01)

# Phones - Apple
vol_port_sp_apple_model = Prophet(holidays = holidays,
                                  changepoint_prior_scale = 0.01,
                                  seasonality_prior_scale = 0.1)
vol_non_port_sp_apple_model = Prophet(holidays = holidays,
                                      changepoint_prior_scale = 0.01,
                                      seasonality_prior_scale = 0.1)
invol_sp_apple_model = Prophet(holidays = holidays,
                               changepoint_prior_scale = 0.5,
                               seasonality_prior_scale = 0.1)

# Tablets
vol_port_tablet_model = Prophet(holidays = holidays)
vol_non_port_tablet_model = Prophet(holidays = holidays)
invol_tablet_model = Prophet(holidays = holidays)

# Wearables - Watches
vol_port_watch_model = Prophet(holidays = holidays)
vol_non_port_watch_model = Prophet(holidays = holidays)
invol_watch_model = Prophet(holidays = holidays)

# Other - MBB
vol_port_mbb_model = Prophet(holidays = holidays)
vol_non_port_mbb_model = Prophet(holidays = holidays)
invol_mbb_model = Prophet(holidays = holidays)

# Other - HPC
vol_port_hpc_model = Prophet(holidays = holidays)
vol_non_port_hpc_model = Prophet(holidays = holidays)
invol_hpc_model = Prophet(holidays = holidays)

# Other - Hum
vol_port_hum_model = Prophet(holidays = holidays)
vol_non_port_hum_model = Prophet(holidays = holidays)
invol_hum_model = Prophet(holidays = holidays)

# Other - Gizmo
vol_port_gizmo_model = Prophet(holidays = holidays)
vol_non_port_gizmo_model = Prophet(holidays = holidays)
invol_gizmo_model = Prophet(holidays = holidays)

# Other - Other
vol_port_other_model = Prophet(holidays = holidays)
vol_non_port_other_model = Prophet(holidays = holidays)
invol_other_model = Prophet(holidays = holidays)

In [134]:
# Fit the models
# Phones - Basic CDMA
vol_port_basic_cdma_model.fit(vol_port_basic_cdma)
vol_non_port_basic_cdma_model.fit(vol_non_port_basic_cdma)
invol_basic_cdma_model.fit(invol_basic_cdma)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_model.fit(vol_port_basic_non_cdma)
vol_non_port_basic_non_cdma_model.fit(vol_non_port_basic_non_cdma)
invol_basic_non_cdma_model.fit(invol_basic_non_cdma)

# Phones - Android & Other
vol_port_sp_android_other_model.fit(vol_port_sp_android_other)
vol_non_port_sp_android_other_model.fit(vol_non_port_sp_android_other)
invol_sp_android_other_model.fit(invol_sp_android_other)

# Phones - Apple
vol_port_sp_apple_model.fit(vol_port_sp_apple)
vol_non_port_sp_apple_model.fit(vol_non_port_sp_apple)
invol_sp_apple_model.fit(invol_sp_apple)

# Tablets
vol_port_tablet_model.fit(vol_port_tablet)
vol_non_port_tablet_model.fit(vol_non_port_tablet)
invol_tablet_model.fit(invol_tablet)

# Wearables - Watches
vol_port_watch_model.fit(vol_port_watch)
vol_non_port_watch_model.fit(vol_non_port_watch)
invol_watch_model.fit(invol_watch)

# Other - MBB
vol_port_mbb_model.fit(vol_port_mbb)
vol_non_port_mbb_model.fit(vol_non_port_mbb)
invol_mbb_model.fit(invol_mbb)

# Other - HPC
vol_port_hpc_model.fit(vol_port_hpc)
vol_non_port_hpc_model.fit(vol_non_port_hpc)
invol_hpc_model.fit(invol_hpc)

# Other - Hum
vol_port_hum_model.fit(vol_port_hum)
vol_non_port_hum_model.fit(vol_non_port_hum)
invol_hum_model.fit(invol_hum)

# Other - Gizmo
vol_port_gizmo_model.fit(vol_port_gizmo)
vol_non_port_gizmo_model.fit(vol_non_port_gizmo)
invol_gizmo_model.fit(invol_gizmo)

# Other - Other
vol_port_other_model.fit(vol_port_other)
vol_non_port_other_model.fit(vol_non_port_other)
invol_other_model.fit(invol_other)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [135]:
# Number of periods into the future to forecast
n = 129

In [136]:
# Create dataframes to hold predictions
# Phones - Basic CDMA
vol_port_basic_cdma_future = vol_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_cdma_future = vol_non_port_basic_cdma_model.make_future_dataframe(periods = n)
invol_basic_cdma_future = invol_basic_cdma_model.make_future_dataframe(periods = n)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_future = vol_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_non_cdma_future = vol_non_port_basic_non_cdma_model.make_future_dataframe(periods = n)
invol_basic_non_cdma_future = invol_basic_non_cdma_model.make_future_dataframe(periods = n)

# Phones - Android & Other
vol_port_sp_android_other_future = vol_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_non_port_sp_android_other_future = vol_non_port_sp_android_other_model.make_future_dataframe(periods = n)
invol_sp_android_other_future = invol_sp_android_other_model.make_future_dataframe(periods = n)

# Phones - Apple
vol_port_sp_apple_future = vol_port_sp_apple_model.make_future_dataframe(periods = n)
vol_non_port_sp_apple_future = vol_non_port_sp_apple_model.make_future_dataframe(periods = n)
invol_sp_apple_future = invol_sp_apple_model.make_future_dataframe(periods = n)

# Tablets
vol_port_tablet_future = vol_port_tablet_model.make_future_dataframe(periods = n)
vol_non_port_tablet_future = vol_non_port_tablet_model.make_future_dataframe(periods = n)
invol_tablet_future = invol_tablet_model.make_future_dataframe(periods = n)

# Wearables - Watches
vol_port_watch_future = vol_port_watch_model.make_future_dataframe(periods = n)
vol_non_port_watch_future = vol_non_port_watch_model.make_future_dataframe(periods = n)
invol_watch_future = invol_watch_model.make_future_dataframe(periods = n)

# Other - MBB
vol_port_mbb_future = vol_port_mbb_model.make_future_dataframe(periods = n)
vol_non_port_mbb_future = vol_non_port_mbb_model.make_future_dataframe(periods = n)
invol_mbb_future = invol_mbb_model.make_future_dataframe(periods = n)

# Other - HPC
vol_port_hpc_future = vol_port_hpc_model.make_future_dataframe(periods = n)
vol_non_port_hpc_future = vol_non_port_hpc_model.make_future_dataframe(periods = n)
invol_hpc_future = invol_hpc_model.make_future_dataframe(periods = n)

# Other - Hum
vol_port_hum_future = vol_port_hum_model.make_future_dataframe(periods = n)
vol_non_port_hum_future = vol_non_port_hum_model.make_future_dataframe(periods = n)
invol_hum_future = invol_hum_model.make_future_dataframe(periods = n)

# Other - Gizmo
vol_port_gizmo_future = vol_port_gizmo_model.make_future_dataframe(periods = n)
vol_non_port_gizmo_future = vol_non_port_gizmo_model.make_future_dataframe(periods = n)
invol_gizmo_future = invol_gizmo_model.make_future_dataframe(periods = n)

# Other - Other
vol_port_other_future = vol_port_other_model.make_future_dataframe(periods = n)
vol_non_port_other_future = vol_non_port_other_model.make_future_dataframe(periods = n)
invol_other_future = invol_other_model.make_future_dataframe(periods = n)

In [137]:
# Make predictions
# Phones - Basic CDMA
vol_port_basic_cdma_forecast = vol_port_basic_cdma_model.predict(vol_port_basic_cdma_future)
vol_non_port_basic_cdma_forecast = vol_non_port_basic_cdma_model.predict(vol_non_port_basic_cdma_future)
invol_basic_cdma_forecast = invol_basic_cdma_model.predict(invol_basic_cdma_future)

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_forecast = vol_port_basic_non_cdma_model.predict(vol_port_basic_non_cdma_future)
vol_non_port_basic_non_cdma_forecast = vol_non_port_basic_non_cdma_model.predict(vol_non_port_basic_non_cdma_future)
invol_basic_non_cdma_forecast = invol_basic_non_cdma_model.predict(invol_basic_non_cdma_future)

# Phones - Android & Other
vol_port_sp_android_other_forecast = vol_port_sp_android_other_model.predict(vol_port_sp_android_other_future)
vol_non_port_sp_android_other_forecast = vol_non_port_sp_android_other_model.predict(vol_non_port_sp_android_other_future)
invol_sp_android_other_forecast = invol_sp_android_other_model.predict(invol_sp_android_other_future)

# Phones - Apple
vol_port_sp_apple_forecast = vol_port_sp_apple_model.predict(vol_port_sp_apple_future)
vol_non_port_sp_apple_forecast = vol_non_port_sp_apple_model.predict(vol_non_port_sp_apple_future)
invol_sp_apple_forecast = invol_sp_apple_model.predict(invol_sp_apple_future)

# Tablets
vol_port_tablet_forecast = vol_port_tablet_model.predict(vol_port_tablet_future)
vol_non_port_tablet_forecast = vol_non_port_tablet_model.predict(vol_non_port_tablet_future)
invol_tablet_forecast = invol_tablet_model.predict(invol_tablet_future)

# Wearables - Watches
vol_port_watch_forecast = vol_port_watch_model.predict(vol_port_watch_future)
vol_non_port_watch_forecast = vol_non_port_watch_model.predict(vol_non_port_watch_future)
invol_watch_forecast = invol_watch_model.predict(invol_watch_future)

# Other - MBB
vol_port_mbb_forecast = vol_port_mbb_model.predict(vol_port_mbb_future)
vol_non_port_mbb_forecast = vol_non_port_mbb_model.predict(vol_non_port_mbb_future)
invol_mbb_forecast = invol_mbb_model.predict(invol_mbb_future)

# Other - HPC
vol_port_hpc_forecast = vol_port_hpc_model.predict(vol_port_hpc_future)
vol_non_port_hpc_forecast = vol_non_port_hpc_model.predict(vol_non_port_hpc_future)
invol_hpc_forecast = invol_hpc_model.predict(invol_hpc_future)

# Other - Hum
vol_port_hum_forecast = vol_port_hum_model.predict(vol_port_hum_future)
vol_non_port_hum_forecast = vol_non_port_hum_model.predict(vol_non_port_hum_future)
invol_hum_forecast = invol_hum_model.predict(invol_hum_future)

# Other - Gizmo
vol_port_gizmo_forecast = vol_port_gizmo_model.predict(vol_port_gizmo_future)
vol_non_port_gizmo_forecast = vol_non_port_gizmo_model.predict(vol_non_port_gizmo_future)
invol_gizmo_forecast = invol_gizmo_model.predict(invol_gizmo_future)

# Other - Other
vol_port_other_forecast = vol_port_other_model.predict(vol_port_other_future)
vol_non_port_other_forecast = vol_non_port_other_model.predict(vol_non_port_other_future)
invol_other_forecast = invol_other_model.predict(invol_other_future)

In [138]:
# Get the datestamp and prediction columns
# Phones - Basic CDMA
vol_port_basic_cdma_forecast_subset = vol_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_CDMA'})
vol_non_port_basic_cdma_forecast_subset = vol_non_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_CDMA'})
invol_basic_cdma_forecast_subset = invol_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_CDMA'})

# Phones - Basic Non CDMA
vol_port_basic_non_cdma_forecast_subset = vol_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_non_CDMA'})
vol_non_port_basic_non_cdma_forecast_subset = vol_non_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_non_CDMA'})
invol_basic_non_cdma_forecast_subset = invol_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_non_CDMA'})

# Phones - Android & Other
vol_port_sp_android_other_forecast_subset = vol_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Android_Other'})
vol_non_port_sp_android_other_forecast_subset = vol_non_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Android_Other'})
invol_sp_android_other_forecast_subset = invol_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Android_Other'})

# Phones - Apple
vol_port_sp_apple_forecast_subset = vol_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Apple'})
vol_non_port_sp_apple_forecast_subset = vol_non_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Apple'})
invol_sp_apple_forecast_subset = invol_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Apple'})

# Tablets
vol_port_tablet_forecast_subset = vol_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Tablet'})
vol_non_port_tablet_forecast_subset = vol_non_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Tablet'})
invol_tablet_forecast_subset = invol_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Tablet'})

# Wearables - Watches
vol_port_watch_forecast_subset = vol_port_watch_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Watch'})
vol_non_port_watch_forecast_subset = vol_non_port_watch_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Watch'})
invol_watch_forecast_subset = invol_watch_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Watch'})

# Other - MBB
vol_port_mbb_forecast_subset = vol_port_mbb_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_MBB'})
vol_non_port_mbb_forecast_subset = vol_non_port_mbb_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_MBB'})
invol_mbb_forecast_subset = invol_mbb_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_MBB'})

# Other - HPC
vol_port_hpc_forecast_subset = vol_port_hpc_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_HPC'})
vol_non_port_hpc_forecast_subset = vol_non_port_hpc_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_HPC'})
invol_hpc_forecast_subset = invol_hpc_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_HPC'})

# Other - Hum
vol_port_hum_forecast_subset = vol_port_hum_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Hum'})
vol_non_port_hum_forecast_subset = vol_non_port_hum_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Hum'})
invol_hum_forecast_subset = invol_hum_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Hum'})

# Other - Gizmo
vol_port_gizmo_forecast_subset = vol_port_gizmo_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Gizmo'})
vol_non_port_gizmo_forecast_subset = vol_non_port_gizmo_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Gizmo'})
invol_gizmo_forecast_subset = invol_gizmo_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Gizmo'})

# Other - Other
vol_port_other_forecast_subset = vol_port_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Other'})
vol_non_port_other_forecast_subset = vol_non_port_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Other'})
invol_other_forecast_subset = invol_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Other'})

In [139]:
# Check the format of the subsetted forecasts (testing only)
invol_tablet_forecast_subset.tail()

,ds,Invol_Tablet
1487,2020-12-27,103.932790
1488,2020-12-28,74.660645
1489,2020-12-29,209.579868
1490,2020-12-30,310.697131
1491,2020-12-31,335.174752


In [140]:
# Growth multiplier
gm = 1.00

In [141]:
# Replace any negative values and scale the forecast using the growth multiplier
# Voluntary Port Outs - Phones
vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'] = vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'].abs() * gm
vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'] = vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'].abs() * gm
vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'] = vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'].abs() * gm
vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'] = vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'].abs() * gm

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'] = vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'].abs() * gm
vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'] = vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'].abs() * gm
vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'] = vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'].abs() * gm
vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'] = vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'].abs() * gm

# Involuntary - Phones
invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'] = invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'].abs() * gm
invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'] = invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'].abs() * gm
invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'] = invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'].abs() * gm
invol_sp_apple_forecast_subset['Invol_SP_Apple'] = invol_sp_apple_forecast_subset['Invol_SP_Apple'].abs() * gm

# Tablet
vol_port_tablet_forecast_subset['Vol_Port_Tablet'] = vol_port_tablet_forecast_subset['Vol_Port_Tablet'].abs() * gm
vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'] = vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'].abs() * gm
invol_tablet_forecast_subset['Invol_Tablet'] = invol_tablet_forecast_subset['Invol_Tablet'].abs() * gm

# Wearables - Watches
vol_port_watch_forecast_subset['Vol_Port_Watch'] = vol_port_watch_forecast_subset['Vol_Port_Watch'].abs() * gm
vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'] = vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'].abs() * gm
invol_watch_forecast_subset['Invol_Watch'] = invol_watch_forecast_subset['Invol_Watch'].abs() * gm

# Other - MBB
vol_port_mbb_forecast_subset['Vol_Port_MBB'] = vol_port_mbb_forecast_subset['Vol_Port_MBB'].abs() * gm
vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'] = vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'].abs() * gm
invol_mbb_forecast_subset['Invol_MBB'] = invol_mbb_forecast_subset['Invol_MBB'].abs() * gm

# Other - HPC
vol_port_hpc_forecast_subset['Vol_Port_HPC'] = vol_port_hpc_forecast_subset['Vol_Port_HPC'].abs() * gm
vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'] = vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'].abs() * gm
invol_hpc_forecast_subset['Invol_HPC'] = invol_hpc_forecast_subset['Invol_HPC'].abs() * gm

# Other - Hum
vol_port_hum_forecast_subset['Vol_Port_Hum'] = vol_port_hum_forecast_subset['Vol_Port_Hum'].abs() * gm
vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'] = vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'].abs() * gm
invol_hum_forecast_subset['Invol_Hum'] = invol_hum_forecast_subset['Invol_Hum'].abs() * gm

# Other - Gizmo
vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'] = vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'].abs() * gm
vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'] = vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'].abs() * gm
invol_gizmo_forecast_subset['Invol_Gizmo'] = invol_gizmo_forecast_subset['Invol_Gizmo'].abs() * gm

# Other - Other
vol_port_other_forecast_subset['Vol_Port_Other'] = vol_port_other_forecast_subset['Vol_Port_Other'].abs() * gm
vol_non_port_other_forecast_subset['Vol_Non_Port_Other'] = vol_non_port_other_forecast_subset['Vol_Non_Port_Other'].abs() * gm
invol_other_forecast_subset['Invol_Other'] = invol_other_forecast_subset['Invol_Other'].abs() * gm

In [142]:
# Check the format of the adjusted forecasts (testing only)
invol_tablet_forecast_subset.tail()

,ds,Invol_Tablet
1487,2020-12-27,103.932790
1488,2020-12-28,74.660645
1489,2020-12-29,209.579868
1490,2020-12-30,310.697131
1491,2020-12-31,335.174752


In [143]:
# Combine the forecasts into one table
# Voluntary Port Outs - Phones
discos_device_grp_pred_table = vol_port_basic_cdma_forecast_subset
discos_device_grp_pred_table.insert(2, 'Vol_Port_Basic_non_CDMA', vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(3, 'Vol_Port_SP_Android_Other', vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(4, 'Vol_Port_SP_Apple', vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'])

# Voluntary Non Port Outs - Phones
discos_device_grp_pred_table.insert(5, 'Vol_Non_Port_Basic_CDMA', vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'])
discos_device_grp_pred_table.insert(6, 'Vol_Non_Port_Basic_non_CDMA', vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(7, 'Vol_Non_Port_SP_Android_Other', vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(8, 'Vol_Non_Port_SP_Apple', vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'])

# Involuntary - Phones
discos_device_grp_pred_table.insert(9, 'Invol_Basic_CDMA', invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'])
discos_device_grp_pred_table.insert(10, 'Invol_Basic_non_CDMA', invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(11, 'Invol_SP_Android_Other', invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'])
discos_device_grp_pred_table.insert(12, 'Invol_SP_Apple', invol_sp_apple_forecast_subset['Invol_SP_Apple'])

# Tablet
discos_device_grp_pred_table.insert(13, 'Vol_Port_Tablet', vol_port_tablet_forecast_subset['Vol_Port_Tablet'])
discos_device_grp_pred_table.insert(14, 'Vol_Non_Port_Tablet', vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'])
discos_device_grp_pred_table.insert(15, 'Invol_Tablet', invol_tablet_forecast_subset['Invol_Tablet'])

# Wearables - Watch
discos_device_grp_pred_table.insert(16, 'Vol_Port_Watch', vol_port_watch_forecast_subset['Vol_Port_Watch'])
discos_device_grp_pred_table.insert(17, 'Vol_Non_Port_Watch', vol_non_port_watch_forecast_subset['Vol_Non_Port_Watch'])
discos_device_grp_pred_table.insert(18, 'Invol_Watch', invol_watch_forecast_subset['Invol_Watch'])

# Other - MBB
discos_device_grp_pred_table.insert(19, 'Vol_Port_MBB', vol_port_mbb_forecast_subset['Vol_Port_MBB'])
discos_device_grp_pred_table.insert(20, 'Vol_Non_Port_MBB', vol_non_port_mbb_forecast_subset['Vol_Non_Port_MBB'])
discos_device_grp_pred_table.insert(21, 'Invol_MBB', invol_mbb_forecast_subset['Invol_MBB'])

# Other - HPC
discos_device_grp_pred_table.insert(22, 'Vol_Port_HPC', vol_port_hpc_forecast_subset['Vol_Port_HPC'])
discos_device_grp_pred_table.insert(23, 'Vol_Non_Port_HPC', vol_non_port_hpc_forecast_subset['Vol_Non_Port_HPC'])
discos_device_grp_pred_table.insert(24, 'Invol_HPC', invol_hpc_forecast_subset['Invol_HPC'])

# Other - Hum
discos_device_grp_pred_table.insert(25, 'Vol_Port_Hum', vol_port_hum_forecast_subset['Vol_Port_Hum'])
discos_device_grp_pred_table.insert(26, 'Vol_Non_Port_Hum', vol_non_port_hum_forecast_subset['Vol_Non_Port_Hum'])
discos_device_grp_pred_table.insert(27, 'Invol_Hum', invol_hum_forecast_subset['Invol_Hum'])

# Other - Gizmo
discos_device_grp_pred_table.insert(28, 'Vol_Port_Gizmo', vol_port_gizmo_forecast_subset['Vol_Port_Gizmo'])
discos_device_grp_pred_table.insert(29, 'Vol_Non_Port_Gizmo', vol_non_port_gizmo_forecast_subset['Vol_Non_Port_Gizmo'])
discos_device_grp_pred_table.insert(30, 'Invol_Gizmo', invol_gizmo_forecast_subset['Invol_Gizmo'])

# Other - Other
discos_device_grp_pred_table.insert(31, 'Vol_Port_Other', vol_port_other_forecast_subset['Vol_Port_Other'])
discos_device_grp_pred_table.insert(32, 'Vol_Non_Port_Other', vol_non_port_other_forecast_subset['Vol_Non_Port_Other'])
discos_device_grp_pred_table.insert(33, 'Invol_Other', invol_other_forecast_subset['Invol_Other'])

In [144]:
# Check the format of the final predictions table (testing only)
discos_device_grp_pred_table.tail()

,ds,Vol_Port_Basic_CDMA,Vol_Port_Basic_non_CDMA,Vol_Port_SP_Android_Other,Vol_Port_SP_Apple,Vol_Non_Port_Basic_CDMA,Vol_Non_Port_Basic_non_CDMA,Vol_Non_Port_SP_Android_Other,Vol_Non_Port_SP_Apple,Invol_Basic_CDMA,Invol_Basic_non_CDMA,Invol_SP_Android_Other,Invol_SP_Apple,Vol_Port_Tablet,Vol_Non_Port_Tablet,Invol_Tablet,Vol_Port_Watch,Vol_Non_Port_Watch,Invol_Watch,Vol_Port_MBB,Vol_Non_Port_MBB,Invol_MBB,Vol_Port_HPC,Vol_Non_Port_HPC,Invol_HPC,Vol_Port_Hum,Vol_Non_Port_Hum,Invol_Hum,Vol_Port_Gizmo,Vol_Non_Port_Gizmo,Invol_Gizmo,Vol_Port_Other,Vol_Non_Port_Other,Invol_Other
1487,2020-12-27,44.627722,18.561743,425.910616,874.799933,18.090951,151.460278,508.120547,867.259861,21.832645,49.575460,88.021899,22.613415,3.494356,108.157327,103.932790,3.560590,232.612899,54.396744,0.926730,3.758397,94.282343,3.247966,1.669530,25.650803,1.023904,44.960374,79.615200,2.363181,101.422195,16.719610,2.312037,14.629398,17.010628
1488,2020-12-28,77.915669,25.508285,556.946337,1036.573678,609.141397,240.415490,1256.993080,1689.279853,20.526158,41.146877,118.015457,56.592211,7.975946,1198.757207,74.660645,4.593342,370.709123,71.221932,1.568195,400.647818,77.113954,10.985023,136.789412,19.877335,0.998356,223.459924,70.811911,2.613994,169.021384,13.146833,2.457492,36.606879,15.660348
1489,2020-12-29,76.220344,25.303908,535.898119,998.147424,546.633205,239.883236,1186.779800,1609.050988,15.153202,41.117659,37.757873,155.778695,8.343071,1048.567677,209.579868,4.298421,361.060718,43.958652,1.500087,348.867386,106.747467,12.093021,118.041418,19.276273,1.186341,209.564293,45.917923,2.359629,160.412497,16.125545,2.811085,35.016975,17.671244
1490,2020-12-30,78.584321,25.574409,557.156746,1023.937662,503.901808,232.657777,1147.340319,1545.080976,54.672922,45.501146,217.997339,325.809637,8.476952,972.907631,310.697131,4.074446,350.780260,28.726108,1.675519,326.954172,127.100167,12.827912,107.316849,37.046234,1.148597,197.347304,37.111906,2.341828,153.676701,27.524053,2.669487,33.440972,18.276946
1491,2020-12-31,78.696159,25.338408,561.535085,1010.523236,494.658190,236.234070,1140.399140,1536.123746,68.704268,57.033998,212.231277,359.937978,8.616239,963.259858,335.174752,4.303729,353.367500,19.221813,1.542939,334.153200,144.597821,12.693111,108.500772,45.236918,1.295838,198.423887,19.662343,2.297597,153.375496,33.171421,2.600146,33.675044,19.327598


In [145]:
# Calculate the root mean squared error (testing only)
print("Vol Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_CDMA'] - fact_table['vol_port_basic_cdma']) ** 2)))
print("Vol Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_non_CDMA'] - fact_table['vol_port_basic_non_cdma']) ** 2)))
print("Vol Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Android_Other'] - fact_table['vol_port_sp_android_other']) ** 2)))
print("Vol Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Apple'] - fact_table['vol_port_sp_apple']) ** 2)))
print("Vol Non Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_CDMA'] - fact_table['vol_non_port_basic_cdma']) ** 2)))
print("Vol Non Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_non_CDMA'] - fact_table['vol_non_port_basic_non_cdma']) ** 2)))
print("Vol Non Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Android_Other'] - fact_table['vol_non_port_sp_android_other']) ** 2)))
print("Vol Non Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Apple'] - fact_table['vol_non_port_sp_apple']) ** 2)))
print("Invol Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Basic_CDMA'] - fact_table['invol_basic_cdma']) ** 2)))
print("Invol Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Basic_non_CDMA'] - fact_table['invol_basic_non_cdma']) ** 2)))
print("Invol SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_SP_Android_Other'] - fact_table['invol_sp_android_other']) ** 2)))
print("Invol SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_SP_Apple'] - fact_table['invol_sp_apple']) ** 2)))
print("Vol Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Tablet'] - fact_table['vol_port_tablet']) ** 2)))
print("Vol Non Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Tablet'] - fact_table['vol_non_port_tablet']) ** 2)))
print("Invol Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Tablet'] - fact_table['invol_tablet']) ** 2)))

Vol Port Basic CDMA RMSE: 27.925002
Vol Port Basic Non CDMA RMSE: 4.409989
Vol Port SP Android Other RMSE: 125.868145
Vol Port SP Apple RMSE: 195.418882
Vol Non Port Basic CDMA RMSE: 97.060096
Vol Non Port Basic Non CDMA RMSE: 26.068003
Vol Non Port SP Android Other RMSE: 121.210936
Vol Non Port SP Apple RMSE: 159.573886
Invol Basic CDMA RMSE: 54.585237
Invol Basic Non CDMA RMSE: 17.743425
Invol SP Android Other RMSE: 412.012581
Invol SP Apple RMSE: 448.472096
Vol Port Tablet RMSE: 5.771944
Vol Non Port Tablet RMSE: 213.296699
Invol Tablet RMSE: 317.634071


In [147]:
# Save the predictions table to a csv file
discos_device_grp_pred_table.to_csv('drive/My Drive/Churn/discos_device_grp_same_day_sep_pred_table.csv')